In [ ]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.research.experimental import continuous_future, history

## 銘柄毎に複数限月のヒストリカルデータを取得

In [ ]:
def get_hist(symbol, start, end, num_of_maturities):
        df = history(
            [continuous_future(symbol,
                               offset=i,
                               adjustment=None) for i in range(num_of_maturities)],
            fields='price', 
            frequency='daily', 
            start=start, 
            end=end
        )
        return df

## 取得銘柄，並びに全ペアを作る

In [ ]:
symbols = ['CL', 'HO', 'XB', 'NG']
symbols_pairs = list(itertools.combinations(symbols, 2))

## 各銘柄のヒストリカルデータを取得

In [ ]:
hist_df = [get_hist(symbol,
                    '2015-01-01', '2018-05-31', 5) for symbol in symbols]

### 一つ表示

In [ ]:
hist_df[0].head()

## 各限月毎にDataFrame化

In [ ]:
maturities_df = [pd.concat(
    [df.iloc[:, i] for df in hist_df], axis=1)
                 for i, x in enumerate(hist_df[0].columns)]

In [ ]:
maturities_df[0].head()

## コラム名をシンボル名（文字列）にする

In [ ]:
for df in maturities_df:
    df.columns = symbols

In [ ]:
maturities_df[0].head()

## 各ペアを比較しプロットする関数

In [ ]:
def plot_ratio(maturity):
    fig = plt.figure(figsize=(14, 14))
    fig.suptitle('offset:{}'.format(maturity))
    ax = [fig.add_subplot(len(symbols_pairs) + 1, 1, i) for i in range(1, len(symbols_pairs) + 1)]
    for i, symbol in enumerate(symbols_pairs):
        ax[i].plot(maturities_df[maturity][list(symbol)].apply(lambda x: x[0] / x[1], axis=1))
        ax[i].set_title('{}/{}'.format(*symbol))

In [ ]:
plot_ratio(0)

In [ ]:
plot_ratio(1)

In [ ]:
plot_ratio(2)

In [ ]:
plot_ratio(3)

In [ ]:
plot_ratio(4)

## 5限月目だけを取り出して詳しく見てみる

In [ ]:
ho_xb_5 = maturities_df[4][['HO', 'XB']]
ho_xb_5_ratio = ho_xb_5['HO'] / ho_xb_5['XB']

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2, sharey=ax1)
ax1.plot(ho_xb_5)
ax2.plot(ho_xb_5_ratio)

In [ ]:
ho_xb_5_ratio.hist(bins=100)

## 簡単なストラテジーとシミュレーション
+ `ho_xb_5_ratio.std()` で全体の標準偏差を取得
+ （1 + ⇑の標準偏差）よりも ratio は高い位置にあるとき，ショートする
+ 10日間ホールド．
+ いい感じじゃないですか＼(^o^)／

In [ ]:
returns = ho_xb_5_ratio.pct_change(10).shift(-10)
short_returns = ((returns[ho_xb_5_ratio > (1 + ho_xb_5_ratio.std())]) * -1).cumsum()
short_returns.plot()

In [ ]:
#ho_xb_5_ratio  > (1 + ho_xb_5_ratio.std())